In [ ]:
!pip install -q Pillow gdown

### Importing required libraries

In [ ]:
import gdown
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import json
import os
import shutil

import torchvision
import numpy as np
import pandas as pd 
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from tqdm.notebook import tqdm
import nltk

In [ ]:
def delete_files(arr):
    for file_path in arr:
        if os.path.exists(file_path):
            os.remove(file_path)

In [ ]:

def delete_folders(arr):
    for folder_path in arr:
        if os.path.exists(folder_path):
            shutil.rmtree(folder_path)

## Data downloading, preprocessing and analyzing

Model

In [ ]:
gdown.download("https://drive.google.com/u/0/uc?export=download&confirm=X2sC&id=1TlNmpLUBw7jJEXgpliy29Am1HpHl-KNJ", "beheaded_inception3.py", True)

'beheaded_inception3.py'

### Downloading the MS COCO Dataset

**Images download**

In [ ]:
%%time
# !wget -q http://images.cocodataset.org/zips/train2017.zip
!wget -q http://images.cocodataset.org/zips/train2014.zip
print("Unzipping...")
# !unzip -q train2017.zip
!unzip -q train2014.zip

Unzipping...
CPU times: user 1.01 s, sys: 210 ms, total: 1.22 s
Wall time: 11min 30s


In [ ]:
delete_files(["/content/train2014.zip"])

**Captions download**

In [ ]:

%%time
# !wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip
print("Unzipping...")
# !unzip -q annotations_trainval2017.zip
!unzip -q annotations_trainval2014.zip

--2021-01-31 08:24:01--  http://images.cocodataset.org/annotations/annotations_trainval2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.108.11
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.108.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252872794 (241M) [application/zip]
Saving to: ‘annotations_trainval2014.zip’

annotations_trainva 100%[===================>] 241.16M  98.7MB/s    in 2.4s    

2021-01-31 08:24:03 (98.7 MB/s) - ‘annotations_trainval2014.zip’ saved [252872794/252872794]

Unzipping...
CPU times: user 30.8 ms, sys: 21.5 ms, total: 52.3 ms
Wall time: 9.82 s


In [ ]:
delete_files([
    "/content/annotations/instances_train2014.json",
    "/content/annotations/instances_val2014.json",
    "/content/annotations/person_keypoints_train2014.json",
    "/content/annotations/person_keypoints_val2014.json",
])

### Data preprocessing

In [ ]:
data = json.load(open("/content/annotations/captions_train2014.json"))

In [ ]:
def data_table(keys):
    dataframes = []

    for key in keys:

        if key == img_key:
            key_value = "file_name"
            key_id = "id"
        elif key == caption_key:
            key_value = "caption"
            key_id = "image_id"

        data_dict = {}
        data_dict[key_value] = []
        data_dict["id"] = []

        for item in data[key]:
            data_dict[key_value].append(item[key_value])
            data_dict["id"].append(item[key_id])

        dataframes.append(pd.DataFrame(data=data_dict))

    return pd.merge(dataframes[0], dataframes[1], on=("id"))

**Get rid of redundant data columns**

In [ ]:
%%time
img_key = "images"
caption_key = "annotations"

df = data_table((img_key, caption_key))

CPU times: user 432 ms, sys: 19.4 ms, total: 452 ms
Wall time: 502 ms


In [ ]:
df.drop(columns="id", inplace=True)
df.rename(columns={"file_name": "image"}, inplace=True)

In [ ]:
df.head()

,image,caption
0,COCO_train2014_000000057870.jpg,A restaurant has modern wooden tables and chairs.
1,COCO_train2014_000000057870.jpg,A long restaurant table with rattan rounded ba...
2,COCO_train2014_000000057870.jpg,a long table with a plant on top of it surroun...
3,COCO_train2014_000000057870.jpg,A long table with a flower arrangement in the ...
4,COCO_train2014_000000057870.jpg,A table is adorned with wooden chairs with blu...


In [ ]:
df = df[df['image'].isin(os.listdir("/content/train2014"))]
df.reset_index(drop=True, inplace=True)

In [ ]:
df

,image,caption
0,COCO_train2014_000000057870.jpg,A restaurant has modern wooden tables and chairs.
1,COCO_train2014_000000057870.jpg,A long restaurant table with rattan rounded ba...
2,COCO_train2014_000000057870.jpg,a long table with a plant on top of it surroun...
3,COCO_train2014_000000057870.jpg,A long table with a flower arrangement in the ...
4,COCO_train2014_000000057870.jpg,A table is adorned with wooden chairs with blu...
...,...,...
414108,COCO_train2014_000000475546.jpg,The patrons enjoy their beverages at the bar.
414109,COCO_train2014_000000475546.jpg,People having a drink in a basement bar.
414110,COCO_train2014_000000475546.jpg,A group of friends enjoys a drink while sittin...
414111,COCO_train2014_000000475546.jpg,Group of people drinking wine at a public loca...


-- Empty table data check --

In [ ]:
df.isna().any()

image      False
caption    False
dtype: bool

## Creating datasets, iterators

In [ ]:
IMAGE_COL_LABEL = df.columns[0]
CAPTION_COL_LABEL = df.columns[1]

In [ ]:
import string

class TextDataset(Dataset):
    
    BOS = "<BOS>"
    EOS = "<EOS>"
    PAD = "<PAD>"

    def __init__(self, text_data: pd.Series, min_word_freq=None, max_word_freq=None, max_sentence_len=20, lower=True):

        if min_word_freq is None:
            self.min_word_freq = -np.inf
        else:
            self.min_word_freq = min_word_freq

        if max_word_freq is None:
            self.max_word_freq = np.inf
        else:
            self.max_word_freq = max_word_freq


        self.max_sentence_len = max_sentence_len
        self.lower = lower

        self.tokenizer = nltk.WordPunctTokenizer()
        self.additional_tokens = set([self.BOS, self.EOS, self.PAD])
        self.token2idx = {self.BOS: 1, self.EOS: 2, self.PAD: 0}

        self.vocab = set(self.append_words(text_data))
        self.token2idx.update({token:num + len(self.additional_tokens) for num, token in enumerate(self.vocab) if token not in self.additional_tokens})

        self.idx2token = {num: token for token, num in self.token2idx.items()}
        self.vocab = self.additional_tokens.union(self.vocab)

    def __len__(self):
        return len(self.vocab)

    def __getitem__(self, text):            
        tokens = self.__tokenize(text)
        text_ids = [token for token in tokens if token in self.vocab]
        text_ids = self.pad_sequence(text_ids)        
        text_ids = [self.token2idx[token] for token in text_ids]
        return text_ids

    def __tokenize(self, row):
        cut_edge = self.max_sentence_len - 2

        row = row.lower() if self.lower else row
        tokens = self.tokenizer.tokenize(row)
        tokens = [token for token in tokens if token not in string.punctuation]
        tokens = tokens[:cut_edge]
        tokens = [self.BOS] + tokens + [self.EOS]
        tokens = self.pad_sequence(tokens)
        return tokens

    def pad_sequence(self, tokens):
        padded_sequence = tokens + (self.max_sentence_len - len(tokens)) * [self.PAD]
        return padded_sequence

    def append_words(self, text: pd.Series):
        counted_words = text.str.lower().str.split().explode().value_counts()
        append_list =  [key for key, val in dict(counted_words).items() 
                        if (val > self.min_word_freq and val < self.max_word_freq) and (key not in self.additional_tokens)]

        return append_list

In [ ]:
%%time
text_data = df.iloc[:, 1]
text_dataset = TextDataset(text_data=text_data, min_word_freq=5, max_sentence_len=20, lower=True)

CPU times: user 3.06 s, sys: 143 ms, total: 3.2 s
Wall time: 3.24 s


### Creating text dataset

### Applying tokenization for data captions

In [ ]:
df[CAPTION_COL_LABEL] = df[CAPTION_COL_LABEL].apply(lambda x: text_dataset[x])

### BatchIterator class for creating data iterators

In [ ]:
class BatchIterator:
    def __init__(self, dataframe, image_col_label, caption_col_label, batch_size, image_transformer, main_img_path, shuffle=False):
        self.dataframe = dataframe
        self.unique_images = np.unique(dataframe[image_col_label])
        self.image_col = image_col_label
        self.caption_col = caption_col_label
        self.image_transformer = image_transformer
        self.main_img_path = main_img_path

        self.num_samples = len(self.unique_images)
        self.batch_size = batch_size
        self.batches_count =  self.num_samples // self.batch_size
        self.shuffle = shuffle

    def __len__(self):
        return self.batches_count

    def __iter__(self):
        indices = np.arange(self.num_samples)

        if self.shuffle:
            np.random.shuffle(indices)

        for start in range(0, self.num_samples, self.batch_size):
            end = min(start + self.batch_size, self.num_samples)

            batch_indices = indices[start:end]
            batch_images = []
            batch_captions = []

            for idx in batch_indices:
                image_name = self.unique_images[idx]

                image = self.__get_image_matrix(image_name)                
                caption = self.__get_caption(image_name)

                batch_images.append(image)
                batch_captions.append(caption)

            yield {
                "images": batch_images,
                "captions": torch.tensor(batch_captions)
            }

    def __get_image_matrix(self, image_name):
        image = Image.open(os.path.join(self.main_img_path, image_name))
        image = image.convert("RGB")
        return self.image_transformer(image)

    def __get_caption(self, image_name):
        all_captions = np.array(
                        self.dataframe[self.dataframe[self.image_col] == image_name][self.caption_col]
                      )
        return random.choice(all_captions)

In [ ]:
def split_data(dataframe, ratios):
    data_len = dataframe.shape[0]

    lengths = [int(data_len * ratio) for ratio in ratios]
    if np.sum(lengths) != data_len:
        lengths[-1] = data_len - np.sum(lengths[:-1])
    
    split_indices = [np.sum(lengths[:i+1]) for i in range(len(lengths))]
    return np.split(df, split_indices)[:len(lengths)]

### Spliting the data

In [ ]:
train_data, valid_data, test_data = split_data(df, [0.8, 0.1, 0.1])

### Creating data iterators

In [ ]:
image_transformer = torchvision.transforms.Compose([
                          torchvision.transforms.ToTensor(),
                          torchvision.transforms.Resize((299, 299)),
                          torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])
                    ])

In [ ]:
BATCH_SIZE = 64
main_img_path = "/content/train2014"

In [ ]:
train_iterator = BatchIterator(train_data, IMAGE_COL_LABEL, CAPTION_COL_LABEL, BATCH_SIZE, image_transformer, main_img_path)
valid_iterator = BatchIterator(valid_data, IMAGE_COL_LABEL, CAPTION_COL_LABEL, BATCH_SIZE, image_transformer, main_img_path)
test_iterator = BatchIterator(test_data, IMAGE_COL_LABEL, CAPTION_COL_LABEL, BATCH_SIZE, image_transformer, main_img_path)

# Main Part

### Models

**ENCODER**

In [ ]:
class Encoder(nn.Module):
    def __init__(self, cnn_model, embedding_size, deny_train_cnn=True):
        super().__init__()
        self.cnn_model = cnn_model

        if deny_train_cnn:
            for param in self.cnn_model.parameters():
                param.requires_grad = False

        self.fc = nn.Linear(self.cnn_model.fc.in_features, embedding_size)

    def forward(self, images):
        _, features, _ = self.cnn_model(images)
        features = self.fc(features)
        return features 

**DECODER**

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, dropout=0.3, num_layers=2, bidirectional=True, cnn_feature_size=2048):
        super().__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_dim % num_directions == 0
        rnn_hidden_dim = hidden_dim // num_directions

        self.vocab_size = vocab_size
        self.embedding_layer = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, rnn_hidden_dim, num_layers=num_layers,
                           dropout=dropout, bidirectional=bidirectional, batch_first=True)
        
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, image_vectors, captions):
        captions = captions[:, :-1]

        embedded = self.embedding_layer(captions)
        concated_data = torch.cat((image_vectors.unsqueeze(1), embedded), dim=1)
        rnn_output, _ = self.rnn(concated_data)
        logits = self.fc(rnn_output)
        return logits 

**SEQ-2-SEQ**

In [ ]:
class CaptionNet(nn.Module):
    def __init__(self, encoder, decoder, device, teacher_forcing_ratio = 0.5):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.teacher_forcing_ratio = teacher_forcing_ratio

    def forward(self, images, captions):
        features = self.encoder(images)
        outputs = self.decoder(features, captions)
        return outputs

In [ ]:
# def caption_image(self, image, vocabulary, max_length=50):
#     result_caption = []

#     with torch.no_grad():
#         x = self.encoderCNN(image).unsqueeze(0)
#         states = None

#         for _ in range(max_length):
#             hiddens, states = self.decoderRNN.lstm(x, states)
#             output = self.decoderRNN.linear(hiddens.squeeze(0))
#             predicted = output.argmax(1)
#             result_caption.append(predicted.item())
#             x = self.decoderRNN.embed(predicted).unsqueeze(0)

#             if vocabulary.itos[predicted.item()] == "<EOS>":
#                 break

#     return [vocabulary.itos[idx] for idx in result_caption]

In [ ]:
    # def sample(self, inputs, states=None, max_len=20):
    #     " accepts pre-processed  image tensor (inputs) and returns predicted sentence (list of tensor ids of length max_len) "
    #     output_sentence = []
    #     for i in range(max_len):
    #         lstm_outputs, states = self.lstm(inputs, states)
    #         lstm_outputs = lstm_outputs.squeeze(1)
    #         out = self.linear(lstm_outputs)
    #         last_pick = out.max(1)[1]
    #         output_sentence.append(last_pick.item())
    #         inputs = self.embedding_layer(last_pick).unsqueeze(1)
        
    #     return output_sentence

In [ ]:
#@title Hyperparameters { run: "auto" }

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

VOCAB_SIZE = len(text_dataset)
EMBEDDING_DIM = 250 #@param {type:"slider", min:100, max:1000, step:50}
HIDDEN_DIM = 256 #@param ["64", "128", "256", "512", "1024", "2048"] {type:"raw"}
DROPOUT = 0.3 #@param {type:"slider", min:0, max:1, step:0.1}
NUM_LAYERS = 2 #@param {type:"slider", min:1, max:10, step:1}
BIDIRECTIONAL = True #@param {type:"boolean"}
TEACHER_FORCE_RATIO = 0.3 #@param {type:"slider", min:0, max:1, step:0.1}

Device:  cuda


Inception model loading

In [ ]:
%%time
from beheaded_inception3 import beheaded_inception_v3
inception = beheaded_inception_v3().train(False)

/usr/local/lib/python3.6/dist-packages/torchvision/models/inception.py:77: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)
Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth



CPU times: user 3min 15s, sys: 773 ms, total: 3min 16s
Wall time: 3min 17s


In [ ]:
encoder = Encoder(inception, EMBEDDING_DIM).to(device)
decoder = Decoder(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, DROPOUT, NUM_LAYERS, BIDIRECTIONAL).to(device)
captionNet = CaptionNet(encoder, decoder, device, TEACHER_FORCE_RATIO).to(device)

In [ ]:
def count_parameters(model):
    return np.sum([param.numel() for param in model.parameters() if param.requires_grad])

print(f'The model has {count_parameters(captionNet):,} trainable parameters')

The model has 6,808,738 trainable parameters


In [ ]:
optimizer = torch.optim.Adam(captionNet.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=0).to(device)
# criterion = nn.CrossEntropyLoss(ignore_index=3)

In [ ]:
#@title {run: "auto" }

EPOCHS = 6 #@param {type:"slider", min:1, max:20, step:1}
CLIP = 2 #@param {type:"slider", min:1, max:10, step:1}

### Training

In [ ]:
def iterate_model(mode, model, iterator, optimizer=None, criterion=None, device=None, clip=None):
    if mode == "train":
        model.train()
    elif mode == "valid":
        model.eval()
    else:
      raise ValueError("Invalid mode, must be 'train' or 'valid'")

    epoch_loss = 0
    epoch_perplexity = 0

    history = []
    for iteration, batch in tqdm(enumerate(iterator), total=len(iterator)):

        images = torch.stack(batch["images"]).to(device)
        captions = batch["captions"].to(device)

        output = model(images, captions)
        loss = criterion(output.view(-1, output.shape[2]), captions.view(-1))

        if mode == "train":
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()

        epoch_loss += loss.item()
        epoch_perplexity += torch.exp(loss)

    epoch_loss /= len(iterator)
    epoch_perplexity /= len(iterator)

    return epoch_loss, epoch_perplexity

In [ ]:
def train(epochs, model, train_iterator, valid_iterator, optimizer, criterion, device, clip, patience, save_epoch):
    MIN_LOSS = np.inf
    CUR_PATIENCE = 0

    for epoch in range(epochs):
            
        train_loss, train_perplexity = iterate_model("train", model, train_iterator, optimizer, criterion, device, clip)
        valid_loss, valid_perplexity = iterate_model("valid", model, valid_iterator, criterion=criterion, device=device)
        
        if valid_loss < MIN_LOSS:
            MIN_LOSS = valid_loss
            best_model = model.state_dict()
        else:
            CUR_PATIENCE += 1
            if CUR_PATIENCE == patience:
                CUR_PATIENCE = 0
                break

        if (epoch + 1) % save_epoch == 0:
            torch.save(best_model, 'best-model.pt')

        print(f"Train loss: {train_loss} | Train perplexity: {train_perplexity}")
        print(f"Valid loss: {valid_loss} | Train perplexity: {valid_perplexity}")
        print("\n")

In [ ]:
PATIENCE = 2
SAVE_EPOCH = 1

In [ ]:
train(EPOCHS, captionNet, train_iterator, valid_iterator, optimizer, criterion, device, CLIP, PATIENCE, SAVE_EPOCH)